# PyTorch quick start guide


In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Device configuration
device = 'cpu'


We can start with MNIST example:

In [2]:
# Transform for preprocessing the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)


We can then define dataset loaders:

In [3]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Networks are defined similar to keras:

In [4]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

We can then create the model and optimizer:

In [5]:
model = NeuralNet().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In contrast with keras, we need to write our training function:

In [6]:
def train_model(model, train_loader, optimizer, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            images, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            if len(labels.size())>1:
                labels = labels.squeeze(-1)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


As well as evaluation function:

In [7]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            if len(labels.size())>1:
                labels = labels.squeeze(-1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

Then, we are ready to train and evaluate our model:

In [8]:
# Train the model:
train_model(model, train_loader, optimizer, criterion, num_epochs=5)


Epoch [1/5], Loss: 0.3970
Epoch [2/5], Loss: 0.1926
Epoch [3/5], Loss: 0.1428
Epoch [4/5], Loss: 0.1148
Epoch [5/5], Loss: 0.0961


In [9]:
evaluate_model(model, test_loader)

Accuracy: 96.84%


We can easily do the same with a Chelo dataset!!

In [10]:
from chelo import DatasetRegistry

dataset = DatasetRegistry.get_dataset("AmesMutagenicityDataset")
dataset.load_data()
dataset_torch = dataset.to_pytorch()
train_set, val_set = torch.utils.data.random_split(dataset_torch, [5000, 764])
train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=val_set, batch_size=64, shuffle=False)


We similarly define the network:

In [11]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(8, 512)
        self.fc2 = nn.Linear( 512, 2)
        self.act = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.dropout(x)
        x = self.softmax(self.fc2(x))
        return x

We can then train the model:

In [12]:
model = NeuralNet().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
train_model(model, train_loader, optimizer, criterion, num_epochs=200)

Epoch [1/200], Loss: 0.6619
Epoch [2/200], Loss: 0.6250
Epoch [3/200], Loss: 0.6223
Epoch [4/200], Loss: 0.6180
Epoch [5/200], Loss: 0.6227
Epoch [6/200], Loss: 0.6187
Epoch [7/200], Loss: 0.6219
Epoch [8/200], Loss: 0.6167
Epoch [9/200], Loss: 0.6175
Epoch [10/200], Loss: 0.6130
Epoch [11/200], Loss: 0.6105
Epoch [12/200], Loss: 0.6122
Epoch [13/200], Loss: 0.6073
Epoch [14/200], Loss: 0.6168
Epoch [15/200], Loss: 0.6147
Epoch [16/200], Loss: 0.6097
Epoch [17/200], Loss: 0.6107
Epoch [18/200], Loss: 0.6090
Epoch [19/200], Loss: 0.6095
Epoch [20/200], Loss: 0.6071
Epoch [21/200], Loss: 0.6076
Epoch [22/200], Loss: 0.6084
Epoch [23/200], Loss: 0.6120
Epoch [24/200], Loss: 0.6161
Epoch [25/200], Loss: 0.6072
Epoch [26/200], Loss: 0.6053
Epoch [27/200], Loss: 0.6050
Epoch [28/200], Loss: 0.6028
Epoch [29/200], Loss: 0.6074
Epoch [30/200], Loss: 0.6052
Epoch [31/200], Loss: 0.6065
Epoch [32/200], Loss: 0.6094
Epoch [33/200], Loss: 0.6030
Epoch [34/200], Loss: 0.6040
Epoch [35/200], Loss: 0

In [14]:
print("Train evaluation:")
evaluate_model(model, train_loader)
print("Test evaluation:")
evaluate_model(model, test_loader)

Train evaluation:
Accuracy: 69.90%
Test evaluation:
Accuracy: 68.19%
